In [2]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load MiDaS model
model_type = "DPT_Large"  # Best quality: use DPT_Hybrid for speed
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.eval()

# Load transforms
transform = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform

# Load and prepare image
image_path = "Top-25-Most-Beautiful-Daisy-Flowers.jpg"
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_tensor = transform(img_rgb).unsqueeze(0)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)
input_tensor = input_tensor.to(device)

# Predict depth
with torch.no_grad():
    prediction = midas(input_tensor)

# Convert to NumPy
depth_map = prediction.squeeze().cpu().numpy()

# Normalize depth map for display/splitting
depth_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# Split depth into 3 regions: foreground, midground, background
foreground_mask = cv2.inRange(depth_normalized, 0, 85)
midground_mask = cv2.inRange(depth_normalized, 86, 170)
background_mask = cv2.inRange(depth_normalized, 171, 255)

# Create 3-layer images
foreground = cv2.bitwise_and(img, img, mask=foreground_mask)
midground = cv2.bitwise_and(img, img, mask=midground_mask)
background = cv2.bitwise_and(img, img, mask=background_mask)

# Save outputs
cv2.imwrite("depth_map.png", depth_normalized)
cv2.imwrite("foreground.png", foreground)
cv2.imwrite("midground.png", midground)
cv2.imwrite("background.png", background)

print("✔ Layers created and saved.")


Using cache found in C:\Users\LENOVO/.cache\torch\hub\intel-isl_MiDaS_master
c:\Users\LENOVO\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LENOVO\miniconda3\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_large_384.pt" to C:\Users\LENOVO/.cache\torch\hub\checkpoints\dpt_large_384.pt
100%|██████████| 1.28G/1.28G [01:50<00:00, 12.5MB/s]
Using cache found in C:\Users\LENOVO/.cache\torch\hub\intel-isl_MiDaS_master


ValueError: too many values to unpack (expected 4)